In [ ]:
pip install sentencepiece


# New section

In [ ]:
pip install keras_metrics

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import keras_metrics
import pandas as pd
import sentencepiece as spm

In [ ]:
df = pd.read_csv('pasangan_baik.csv')
#lines_1 = df['key'].to_list()
#lines_2 = df['value'].to_list()

In [ ]:
with open('min-ind.txt','r',encoding='utf-8') as f:
  lines = f.read().split('\n')
with open('min-ind_train.txt','w',encoding='utf-8') as w:
  for line in lines:
    w.write(line+'\n')
lines_1 = []
lines_2 = []
with open('min-ind_train.txt','r',encoding='utf-8') as min:
  for line in min.readlines():
    split = line.split(',')
    if split!=['\n']:

      lines_1.append(split[0])
      lines_2.append(split[1])

   #preprocess kata di seitap lines 
#f = lambda x: re.sub('\n$', '', x)
#lines_1 = [f(x) for x in lines_1]
#lines_2 = [f(x) for x in lines_2]
        
        # preprocess drop duplicates 
#df = pd.DataFrame([lines_1, lines_2], index=['key', 'value']).T.drop_duplicates()
#lines_1 = df['key'].to_list()
#lines_2 = df['value'].to_list()
with open('min-ind_input.txt','w',encoding='utf-8') as enc_txt:
  for line in lines_1:
    enc_txt.write(line+'\n')
with open('min-ind_target.txt','w',encoding='utf-8') as tar_txt:
  for line in lines_2:
    tar_txt.write(line)

In [ ]:
import pandas as pd
import re
from tqdm import tqdm

def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value
kamus = {}
for i, row in tqdm(df.iterrows()):
  key = row['key']
  append_value(kamus, key, row['value'])

13207it [00:01, 7434.75it/s]


In [ ]:
list_lines = [s.split(',') for s in lines]
list_lines

In [ ]:
kamus

In [ ]:
dict_ = {}
for s in list_lines:
  if len(s)!=1:
    dict_[s[0]] = s[1]
dict_

In [ ]:
temp_dict = {}
for key,value in dict_.items():
  if key in temp_dict:
    temp_dict[key].append(value)
  else:
    temp_dict[key] = value

In [ ]:
# Source words with multiple translations in dictionary
temp_dict

In [ ]:
#import sentencepiece as spm
model_inp = spm.SentencePieceTrainer.train("--input=min-ind_input.txt --model_prefix=input --vocab_size=100")
sp_inp = spm.SentencePieceProcessor()
sp_inp.load('input.model')
model_tar = spm.SentencePieceTrainer.train("--input=min-ind_target.txt --model_prefix=target --vocab_size=100")
sp_tar = spm.SentencePieceProcessor()
sp_tar.load('target.model')

True

In [ ]:
### Prepare data ###
input_encode = []
target_encode = []
with open('min-ind_input.txt','r',encoding='utf-8') as inp:
  for line in inp:
    input_encode.append(sp_inp.encode_as_pieces(line))
with open('min-ind_target.txt','r',encoding='utf-8') as tar:
  for line in tar:
    target_encode.append(['\t']+sp_tar.encode_as_pieces(line)+['\n'])
 

In [ ]:
## Create a dictionary for characters mapping ##
input_subwords = np.unique(np.array([tk for tok in input_encode for tk in tok]))
input_subwords = np.append(input_subwords," ")
target_subwords = np.unique(np.array([tk for tok in target_encode for tk in tok]))
target_subwords = np.append(target_subwords," ")
input_token_index = dict([(char,i) for i,char in enumerate(sorted(list(input_subwords)))])
target_token_index = dict([(char,i) for i,char in enumerate(sorted(list(target_subwords)))])

num_encoder_tokens= len(input_subwords)
num_decoder_tokens = len(target_subwords)

In [ ]:
max_encoder_seq_length = max([len(words) for words in input_encode])
max_decoder_seq_length = max([len(words) for words in target_encode])

encoder_input_data = np.zeros((len(input_encode),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(target_encode),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
 
for i,(enc_word,tar_word) in enumerate(zip(input_encode,target_encode)):
  for t,char in enumerate(enc_word):
    encoder_input_data[i,t,input_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,input_token_index[" "]] = 1.0 # encoding space at the end of encoder word for decoder input to reach to next word for encoding
  for t,char in enumerate(tar_word):
    decoder_input_data[i,t,target_token_index[char]] = 1.0
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]] = 1.0
  decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
  decoder_target_data[i,t:,target_token_index[" "]] = 1.0

In [ ]:
len(encoder_input_data)

13761

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers
seed = 7
np.random.seed(seed)
kfold_index,test_index = train_test_split(list(range(len(encoder_input_data))),test_size=0.2,random_state=seed)
fold_var = 5
kf = KFold(n_splits = fold_var,random_state=seed, shuffle=True)
validation_accuracy = []
validation_loss = []
exac_match_accuracy = []
#validation_recall = []

In [ ]:
encoder_input_data_k = pd.Series(list(encoder_input_data)).iloc[kfold_index].to_numpy()
decoder_input_data_k = pd.Series(list(decoder_input_data)).iloc[kfold_index].to_numpy()
decoder_target_data_k = pd.Series(list(decoder_target_data)).iloc[kfold_index].to_numpy()

encoder_input_data_k = np.array([list(en) for en in encoder_input_data_k])
decoder_input_data_k = np.array([list(en) for en in decoder_input_data_k])
decoder_target_data_k = np.array([list(en) for en in decoder_target_data_k])

encoder_input_data_test = pd.Series(list(encoder_input_data)).iloc[test_index].to_numpy()
decoder_input_data_test = pd.Series(list(decoder_input_data)).iloc[test_index].to_numpy()
decoder_target_data_test = pd.Series(list(decoder_target_data)).iloc[test_index].to_numpy()

encoder_input_data_test = np.array([list(en) for en in encoder_input_data_test])
decoder_input_data_test = np.array([list(en) for en in decoder_input_data_test])
decoder_target_data_test = np.array([list(en) for en in decoder_target_data_test])

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 10 # Number of epochs to train for.
latent_dim_1 = 128
latent_dim_2 = 256
latent_dim_3 = 512 # Latent dimensionality of the encoding space.
num_samples = 13761  # Number of samples to train on.

In [ ]:
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(encoder_input_data_k)))):
  print('Iteration number : ', i+1)

  # Define an input sequence and process it.
  encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
  encoder_outputs,forward_h,forward_c,backward_h,backward_c= keras.layers.Bidirectional(keras.layers.LSTM(latent_dim_2, return_state=True),merge_mode='concat',name='encoder_lstm1')(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  state_h = tf.keras.layers.Concatenate()([forward_h,forward_c])
  state_c = tf.keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [state_h, state_c]

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, return_state=True)(decoder_inputs, initial_state=encoder_states)
  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  decoder_outputs = decoder_dense(decoder_outputs)

  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  encoder_input_data_t = pd.Series(list(encoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data_k)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data_k)).iloc[val_index].to_numpy()

  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])

  x_train = [encoder_input_data_t, decoder_input_data_t]
  y_train = decoder_target_data_t

  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val

  # Compile Model
  model.compile(
      optimizer="adam", loss="categorical_crossentropy", 
      metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  
  # Fitting Model
  model_history = model.fit(
      [encoder_input_data_t, decoder_input_data_t],
      decoder_target_data_t,
      batch_size=batch_size,
      epochs=epochs,
      validation_data = (x_val,y_val)
  )
  # Save model
  model_name = 'model_bilstm_256_{}-Fold-{}'.format(fold_var,i+1)
  model.save(model_name)

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  model_inf = keras.models.load_model(model_name)

  encoder_inputs = model_inf.input[0]  # input_1
  encoder_outputs,state_h_enc,state_c_enc,backward_h,backward_c  = model_inf.layers[1].output  # lstm_1
  states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  states_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model_inf.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model_inf.layers[5]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model_inf.layers[6]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )

  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence

  # Evaluate Model
  results = model.evaluate(x_val, y_val, batch_size=batch_size)
  results = dict(zip(model.metrics_names,results))
  
  # Save Evaluation Result
  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])
  #validation_recall.append(results['recall'])
  
  # Check Result Translation
  list_match = []
  count = 0
  print("Checking Translation Accuracy")
  for seq_index in val_index:
      # Take one sequence (part of the training set)
      # for trying out decoding.
      input_seq = encoder_input_data[seq_index : seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      decoded_sentence = decoded_sentence[1:-1]
      exact_result = decoded_sentence == lines_2[seq_index][:-1]
      # print("-",seq_index)
      # print("Input sentence:", ''.join(lines_1[seq_index]))
      # print("Decoded sentence:", ''.join(decoded_sentence))
      # print("Target sentence:", ''.join(lines_2[seq_index][:-1]))
      # print("Exact Match:", ''.join(str(exact_result)))
      list_match.append(exact_result)
      count+=1
  print("Exact Match count : ", sum(list_match))
  print("Tested Data count : ", count)
  print("Translation Accuracy : ", sum(list_match)/count)
  exac_match_accuracy.append(sum(list_match)/count)

Iteration number :  1
Epoch 1/10
138/138 [==============================] - 61s 43ms/step - loss: 1.6209 - accuracy: 0.6661 - recall_7: 0.5787 - precision_7: 0.9510 - val_loss: 1.3092 - val_accuracy: 0.7024 - val_recall_7: 0.6048 - val_precision_7: 0.9746
Epoch 2/10
138/138 [==============================] - 4s 30ms/step - loss: 1.2010 - accuracy: 0.7227 - recall_7: 0.6755 - precision_7: 0.9733 - val_loss: 1.1388 - val_accuracy: 0.7320 - val_recall_7: 0.6954 - val_precision_7: 0.9648
Epoch 3/10
138/138 [==============================] - 4s 30ms/step - loss: 0.9921 - accuracy: 0.7648 - recall_7: 0.7056 - precision_7: 0.9755 - val_loss: 0.8509 - val_accuracy: 0.7939 - val_recall_7: 0.7105 - val_precision_7: 0.9781
Epoch 4/10
138/138 [==============================] - 4s 30ms/step - loss: 0.6960 - accuracy: 0.8231 - recall_7: 0.7449 - precision_7: 0.9645 - val_loss: 0.6197 - val_accuracy: 0.8429 - val_recall_7: 0.7598 - val_precision_7: 0.9647
Epoch 5/10
138/138 [=========================

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-1/assets


INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-1/assets


35/35 [==============================] - 1s 17ms/step - loss: 0.3455 - accuracy: 0.9195 - recall_7: 0.8899 - precision_7: 0.9560
Checking Translation Accuracy
Exact Match count :  1256
Tested Data count :  2202
Translation Accuracy :  0.5703905540417802
Iteration number :  2
Epoch 1/10
138/138 [==============================] - 11s 42ms/step - loss: 1.6220 - accuracy: 0.6640 - recall_8: 0.5763 - precision_8: 0.9532 - val_loss: 1.2847 - val_accuracy: 0.7113 - val_recall_8: 0.6274 - val_precision_8: 0.9831
Epoch 2/10
138/138 [==============================] - 4s 30ms/step - loss: 1.1916 - accuracy: 0.7241 - recall_8: 0.6769 - precision_8: 0.9728 - val_loss: 1.0793 - val_accuracy: 0.7469 - val_recall_8: 0.6982 - val_precision_8: 0.9804
Epoch 3/10
138/138 [==============================] - 4s 30ms/step - loss: 0.9285 - accuracy: 0.7765 - recall_8: 0.7100 - precision_8: 0.9744 - val_loss: 0.7847 - val_accuracy: 0.8071 - val_recall_8: 0.7193 - val_precision_8: 0.9752
Epoch 4/10
138/138 [====

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-2/assets


...

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-3/assets


35/35 [==============================] - 1s 17ms/step - loss: 0.3411 - accuracy: 0.9197 - recall_9: 0.8951 - precision_9: 0.9535
Checking Translation Accuracy
Exact Match count :  1292
Tested Data count :  2202
Translation Accuracy :  0.5867393278837421
Iteration number :  4
Epoch 1/10
138/138 [==============================] - 11s 45ms/step - loss: 1.6221 - accuracy: 0.6666 - recall_10: 0.5784 - precision_10: 0.9546 - val_loss: 1.2884 - val_accuracy: 0.7163 - val_recall_10: 0.6258 - val_precision_10: 0.9893
Epoch 2/10
138/138 [==============================] - 4s 29ms/step - loss: 1.1982 - accuracy: 0.7236 - recall_10: 0.6779 - precision_10: 0.9733 - val_loss: 1.1168 - val_accuracy: 0.7414 - val_recall_10: 0.6884 - val_precision_10: 0.9859
Epoch 3/10
138/138 [==============================] - 4s 29ms/step - loss: 0.9697 - accuracy: 0.7691 - recall_10: 0.7065 - precision_10: 0.9760 - val_loss: 0.8193 - val_accuracy: 0.7983 - val_recall_10: 0.7225 - val_precision_10: 0.9685
Epoch 4/10
1

INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-4/assets


INFO:tensorflow:Assets written to: model_bilstm_256_5-Fold-4/assets


35/35 [==============================] - 1s 16ms/step - loss: 0.3392 - accuracy: 0.9194 - recall_10: 0.8915 - precision_10: 0.9562
Checking Translation Accuracy


In [ ]:
#Buat kamus dan update input dan target teks
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], list):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

 #preprocess kata di seitap lines 
f = lambda x: re.sub('\n$', '', x)
lines_1 = [f(x) for x in lines_1]
lines_2 = [f(x) for x in lines_2]
        
        # preprocess drop duplicates 
df = pd.DataFrame([lines_1, lines_2], index=['key', 'value']).T.drop_duplicates()
lines_1 = df['key'].to_list()
lines_2 = df['value'].to_list()
kamus = {}

for i, row in tqdm(df.iterrows()):
  key = row['key']
  append_value(kamus, key, row['value'])

In [ ]:
kamus

In [ ]:
# Show Accuracy Result
for i,val_acc in enumerate(validation_accuracy):
  print("Fold {} Accuracy : {}".format(i+1,val_acc))
print("Average Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(validation_accuracy), np.std(validation_accuracy)))

In [ ]:
#Prep Test Data
x_test = [encoder_input_data_test,decoder_input_data_test]
y_test = decoder_target_data_test

# Evaluate Model
results = model.evaluate(x_test, y_test, batch_size=batch_size)
results = dict(zip(model.metrics_names,results))

In [ ]:
seq_index = 100
print(lines_1[seq_index])
print(lines_2[seq_index])

In [ ]:
# Check Result Translation -> Test Data
list_match = []
count = 0
for seq_index in test_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence[1:-1]
    print("-",seq_index)
    print("Input sentence:", ''.join(lines_1[seq_index]))
    print("Decoded sentence:", ''.join(decoded_sentence))
    print("Target sentence:", ''.join(lines_2[seq_index][:-1]))
    exact_result = decoded_sentence in lines_2[seq_index][:-1]
    print("Exact Match:", ''.join(str(exact_result)))
    list_match.append(exact_result)
    count+=1
print("Exact Match count : ", sum(list_match))
print("Tested Data count : ", count)
print("Translation Accuracy : ", sum(list_match)/count)

Streaming output truncated to the last 5000 lines.
Target sentence: merupakan
Exact Match: True
- 7102
Input sentence: eksistensi
Decoded sentence: ekstensi
Target sentence: eksistensi
Exact Match: False
- 5898
Input sentence: dihasilan
Decoded sentence: dihasilkan
Target sentence: dihasilkan
Exact Match: True
- 8139
Input sentence: paluasan
Decoded sentence: perluasan
Target sentence: perluasan
Exact Match: True
- 10969
Input sentence: dimaksud
Decoded sentence: dimaksudkan
Target sentence: dimaksud
Exact Match: False
- 11448
Input sentence: rukyah
Decoded sentence: rukhh
Target sentence: rukyah
Exact Match: False
- 10815
Input sentence: mampromosian
Decoded sentence: mempromosikan
Target sentence: mempromosikan
Exact Match: True
- 10527
Input sentence: panganuiknyo
Decoded sentence: penganutnya
Target sentence: penganutnya
Exact Match: True
- 4886
Input sentence: sekretariat
Decoded sentence: serketariat
Target sentence: sekretariat
Exact Match: False
- 8515
Input sentence: sapandape

In [ ]:
!zip -r model_bilstm_256_5-Fold-5.zip model_bilstm_256_5-Fold-5